In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests

sns.set()

In [ ]:
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'

url_data = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:ÅlderTotA",
        "values": []
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url, json=url_data)
r.status_code

In [ ]:
json = r.json()

In [ ]:
data = json['data']
data[0]

In [ ]:
data_list = []

for rec in data:
    data_list.append((rec['key'][0],rec['key'][1],rec['values'][0]))

In [ ]:
df = pd.DataFrame(data_list)
df.columns=['gender','year','population']
df['population'] = df['population'].astype(int)

In [ ]:
population = df.groupby('year').sum()
population['factor'] = population['population'] / population['population'].shift()
population.loc['2020','population'] = population.loc['2019','population'] * population.loc['2019','factor'] 
population

In [ ]:
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101G/ManadFoddDod'
data = {
  "query": [
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Manad",
      "selection": {
        "filter": "item",
        "values": [
          "01",
          "02",
          "03",
          "04",
          "05",
          "06",
          "07",
          "08",
          "09",
          "10",
          "11",
          "12",
          "90"
        ]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "item",
        "values": [
          "000000NF"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json=data)
r.status_code

In [ ]:
data_list=[]

json = r.json()

data = json['data']

for rec in data:
    data_list.append((rec['key'][0],rec['key'][1],rec['key'][2],rec['values'][0]))

In [ ]:
deaths = pd.DataFrame(data_list)
deaths.columns=['gender','month','year','dead']
deaths['dead'] = deaths['dead'].astype(int)
deaths = deaths.groupby('year').sum()
deaths

In [ ]:
def parse_scb_prel():
    
    def monthname2number(m):
        if m == 'januari' : return 1
        if m == 'februari' : return 2
        if m == 'mars' : return 3
        if m == 'april' : return 4
        if m == 'maj' : return 5
        if m == 'juni' : return 6
        if m == 'juli' : return 7
        if m == 'augusti' : return 8
        if m == 'september' : return 9
        if m == 'oktober' : return 10
        if m == 'november' : return 11
        if m == 'december' : return 12

    df = pd.read_csv('scb-dead_per_day.csv',sep=';',usecols=[1,2,3,4,5,6,7,10,11],thousands=',')
        
    df['month'] = df['Månad'].apply(monthname2number)
    df[['Månad','month']] = df[['Månad','month']].replace(np.nan,0)
    df['month'] = df['month'].astype(int)
    
    df = df.replace(np.nan,0)
    
    df[['2015','2016','2017','2018','2019','2020','2015-2019']] = df[['2015','2016','2017','2018','2019','2020','2015-2019']].astype(int)

    
    unknown_death_day = df.iloc[-1,:7]
    
    df.drop(366,inplace=True)
    
    df = df.loc[df['2020'] != 0]
    
    #df = df.drop('Månad',axis=1)
    
    index = pd.date_range('2020-01-01',periods=len(df))
    df.index = index
    
    known_death_day = df.iloc[:,:7].sum()
    
    year_fraction = len(df) / 366
    
    unknown_death_day[['2015','2016','2017','2018','2019','2015-2019']] = unknown_death_day[
    ['2015','2016','2017','2018','2019','2015-2019']] * year_fraction
    
    dead_pct_per_day = df[['2015','2016','2017','2018','2019','2020','2015-2019']] / known_death_day
    
    all_deaths = df[['2015','2016','2017','2018','2019','2020','2015-2019']] + unknown_death_day * dead_pct_per_day
    
    all_deaths = pd.concat([all_deaths,df['Månad']],axis=1)
    
    return all_deaths
    

In [ ]:
deaths_2020 = parse_scb_prel()['2020']

last_day_nr = deaths_2020.index[-1].dayofyear
last_day_nr

In [ ]:
YTD_deaths = deaths_2020.sum()
YTD_deaths

In [ ]:
full_year_deaths = 1 / (last_day_nr/365) * YTD_deaths
full_year_deaths

In [ ]:
deaths.loc['2020','dead'] = full_year_deaths

In [ ]:
deaths

In [ ]:
historical_pop = population.join(deaths)

In [ ]:
historical_pop.drop('factor',axis=1,inplace=True)
historical_pop

In [ ]:
historical_pop['mortality'] = historical_pop['dead'] / historical_pop['population']
historical_pop

In [ ]:
historical_pop['mortality'].plot(style='ro--',figsize=(18,12),
                                 title='SWEDEN mortality (population adjusted) 1860 - 2020\n'\
                                'ASSUMPTIONS : \n1) Population for 2020 grows as 2018-2019\n'\
                                '2) Deaths full year 2020 grow proportionally as for Y2D\n'\
                                'DataSource : SCB.se')

plt.ylabel('mortality')
plt.savefig('scb_historical_pop_dead.jpg',format='jpg',dpi=400)

In [ ]:
historical_pop.loc['1915' : '1920']